In [1]:
%cd ../..
%load_ext autoreload

%autoreload 2

/home/kirell/work/emv


In [2]:
import os
from pathlib import Path
import emv
import emv.features.pose as pose
import numpy as np

/home/kirell/.cache/pypoetry/virtualenvs/emv-IgKv7S_O-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.1


'4c55dc50355d5e923642c59ad2a23d6ad54711e7'

In [3]:
pose_data = 'data/pifpaf'
videos = '/mnt/d/Videos/EPFL'
%ls $videos

basketball.mp4*  gym.mp4*  running.mp4*  swimming.mp4*  weights.mp4*


In [4]:
data_path = Path(pose_data, 'basketball.jl')
# pose.process_video(pose.PifPafModel.fast, data_path, 
#                    Path(videos, 'basketball.mp4'), skip_frame=10)

In [5]:
data = rts.utils.read_jsonlines_file(data_path)

In [16]:
data = pose.extract_frame_data(data_path, 0.5)

In [17]:
data

{140: {'angles': [[159.87614503290223,
    137.97025297151285,
    173.9082657462902,
    174.455221652199,
    176.32702565484175,
    168.72095466625817,
    172.8623498116419,
    176.27001693997997,
    79.93942524625857,
    99.03416868314714,
    90.27250132642273],
   [None,
    171.7093504833666,
    168.364907483736,
    177.32595456676117,
    173.94527047146644,
    176.72847277310953,
    None,
    175.53113564912857,
    21.737246930578227,
    93.08112235709437,
    80.305616815793],
   [48.574518157659405,
    47.54245527732417,
    None,
    None,
    None,
    178.5892712794159,
    171.2993248743615,
    None,
    99.96255158048795,
    None,
    None],
   [None,
    143.73444391484443,
    None,
    164.75464401856786,
    174.89345215432493,
    171.70698018639908,
    None,
    136.67544413829768,
    21.73223469591333,
    None,
    None],
   [None,
    68.34424605228709,
    None,
    74.1065630405051,
    None,
    75.91784555034019,
    None,
    165.6413692254